In [1]:
# Ensamble model tuning

In [2]:
#!pip install keras-swa

In [3]:
# !pip install scikit-learn==0.21.3 --user

In [4]:
#!pip install lightgbm

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
import os
import matplotlib.pyplot as plt

In [2]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import lightgbm as lgbm

Using TensorFlow backend.


In [3]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, GaussianNoise, Concatenate, LeakyReLU
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint,Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA # swa


# 데이터 불러오기

In [4]:
# 0. 패키지 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. 데이터 불러오기
train = pd.read_csv('train.csv', index_col=0) # 코랩 사용 시 경로(/content/drive/My Drive/Colab Notebooks/) 추가
test = pd.read_csv('test.csv', index_col=0)
sample_submission = pd.read_csv('sample_submission.csv', index_col=0)

# 2. Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 3.모델에 적용할 데이터 셋 준비 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler, Normalizer

train_x = train.drop(columns=['type', 'type_num'], axis=1)
del train_x['fiberID']
train_y = train['type_num']
test_x = test
del test_x['fiberID']

x = train_x.copy() 

down_quantiles = x.quantile(0.002) # 전처리 비율 조정
up_quantiles = x.quantile(0.998)

outliers_low = (x < down_quantiles)
outliers_high = (x > up_quantiles)

x[outliers_low] = np.nan
x.fillna(down_quantiles, inplace=True)

x[outliers_high] = np.nan
x.fillna(up_quantiles, inplace=True)

In [5]:
x = x.values
train_y = train_y.values

x_train, x_test, y_train, y_test = train_test_split(x, train_y, stratify=train_y, train_size=0.9)

# # 3-1 다중 분류 시 One-hot Encoding
# from sklearn.preprocessing import OneHotEncoder
# onehot_encoder = OneHotEncoder()
# y_train = y_train.reshape(len(y_train), -1) 
# y_val = y_val.reshape(len(y_val), -1)
# y_train = onehot_encoder.fit_transform(y_train)
# y_val = onehot_encoder.fit_transform(y_val)

# print(x_train.shape)
# print(y_train.shape)

In [6]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf

class Gelu(Activation):
    def __init__(self, activation, **kwargs):
        super(Gelu, self).__init__(activation, **kwargs)
        self.__name__='gelu'
        
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * np.power(x, 3))))

get_custom_objects().update({'gelu': Gelu(gelu)})

# Scaler

In [7]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

test_x =scaler.transform(test_x)

In [8]:
print(x_train.shape)
print(y_train.shape)

(179991, 20)
(179991,)


# Variables

In [9]:
drop1 = 0.4
drop2 = 0.45
drop3 = 0.5

In [10]:
import datetime
start=datetime.datetime.now()
print(start)

2020-02-24 12:27:10.073881


# Deep Learning Model
- EarlyStop, CosineScheduler, CheckPoint

In [11]:
def model_1(epochs=1200, drop=0.45):

    inps= Input(shape = (20,))
    
    la = Dense(128)(inps)
    la = Dense(256, activation='relu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='relu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='relu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='relu')(la)
    la = Dropout(drop)(la)

    outs=Dense(19, activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=1.e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-08, decay=1e-6),metrics=['accuracy'])
    
    return models


In [12]:
def model_2(epochs=1200, drop=0.45):

    inps= Input(shape = (20,))
    
    la = Dense(128)(inps)
    la = Dense(256, activation='elu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='elu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='elu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='elu')(la)
    la = Dropout(drop)(la)
    outs=Dense(19, activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=1.e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-08, decay=1e-6),metrics=['accuracy'])
    
    return models


In [13]:
def model_3(epochs=1200, drop=0.45):

    inps= Input(shape = (20,))
    
    la = Dense(128)(inps)
    la = Dense(256, activation='selu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='selu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='selu')(la)
    la = Dropout(drop)(la)
    la = Dense(256, activation='selu')(la)
    la = Dropout(drop)(la)
    outs=Dense(19, activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=1.e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-08, decay=1e-6),metrics=['accuracy'])
    
    return models


In [14]:
def model_4(epochs=1200, drop=0.45):

    inps= Input(shape = (20,))
    
    la = Dense(128)(inps)
    la = Dense(256)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = Activation(gelu)(la)
    la = Dropout(drop)(la)
    outs=Dense(19, activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=1.e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-08, decay=1e-6),metrics=['accuracy'])
    
    return models

In [15]:
def model_5(epochs=1200, drop=0.45):

    inps= Input(shape = (20,))
    
    la = Dense(128)(inps)
    la = LeakyReLU(alpha=0.07)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = LeakyReLU(alpha=0.07)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = LeakyReLU(alpha=0.07)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = LeakyReLU(alpha=0.07)(la)
    la = Dropout(drop)(la)
    la = Dense(256)(la)
    la = Dropout(drop)(la)
    outs=Dense(19, activation='softmax',name='output')(la)

    models = Model(inputs=inps, outputs=outs)
    models.summary()
    models.compile(loss="sparse_categorical_crossentropy", \
                   optimizer=Adam(lr=1.e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-08, decay=1e-6),metrics=['accuracy'])
    
    return models

In [16]:
sk_params = { 'validation_split': 1./9. }
nepoch1 = 1000
nepoch2 = 850
nepoch3 = 900

model_1a = KerasClassifier(build_fn = model_1, epochs = nepoch1, drop=drop1, verbose=2, batch_size=256, **sk_params)

model_1b = KerasClassifier(build_fn = model_1, epochs = nepoch2, drop=drop2, verbose=2, batch_size=256, **sk_params)

model_1c = KerasClassifier(build_fn = model_1, epochs = nepoch3, drop=drop3, verbose=2, batch_size=256, **sk_params)

model_2a = KerasClassifier(build_fn = model_2, epochs = nepoch1, drop=drop1, verbose=2, batch_size=256, **sk_params)

model_2b = KerasClassifier(build_fn = model_2, epochs = nepoch2, drop=drop2, verbose=2, batch_size=256, **sk_params)

model_2c = KerasClassifier(build_fn = model_2, epochs = nepoch3, drop=drop3, verbose=2, batch_size=256, **sk_params)

model_3a = KerasClassifier(build_fn = model_3, epochs = nepoch1, drop=drop1, verbose=2, batch_size=256, **sk_params)

model_3b = KerasClassifier(build_fn = model_3, epochs = nepoch2, drop=drop2, verbose=2, batch_size=256, **sk_params)

model_3c = KerasClassifier(build_fn = model_3, epochs = nepoch3, drop=drop3, verbose=2, batch_size=256, **sk_params)

model_4a = KerasClassifier(build_fn = model_4, epochs = nepoch1, drop=drop1, verbose=2, batch_size=256, **sk_params)

model_4b = KerasClassifier(build_fn = model_4, epochs = nepoch2, drop=drop2, verbose=2, batch_size=256, **sk_params)

model_4c = KerasClassifier(build_fn = model_4, epochs = nepoch3, drop=drop3, verbose=2, batch_size=256, **sk_params)

model_5a = KerasClassifier(build_fn = model_5, epochs = nepoch1, drop=drop1, verbose=2, batch_size=256, **sk_params)

model_5b = KerasClassifier(build_fn = model_5, epochs = nepoch2, drop=drop2, verbose=2, batch_size=256, **sk_params)

model_5c = KerasClassifier(build_fn = model_5, epochs = nepoch3, drop=drop3, verbose=2, batch_size=256, **sk_params)



In [17]:
ensemble_clf = VotingClassifier(estimators = [ ('model3', model_1a), ('model4', model_1b), ('model5', model_1c), 
                                             ('model9', model_4a), ('model10', model_4b), ('model11', model_4c)], voting = 'hard')

# ensemble_clf = VotingClassifier(estimators = [ ('model1', model_1a), ],  voting = 'soft')

In [ ]:
ensemble_clf.fit(x_train, y_train)

In [ ]:
print(ensemble_clf.score(x_test, y_test))

In [ ]:
end=datetime.datetime.now()
print("걸린 시간:", end-start)

pred_test=ensemble_clf.predict_proba(test_x)

In [ ]:
print(pred_test.shape)

# Model Predict

In [ ]:
# 제출 파일 생성
submission = pd.DataFrame(data=pred_test, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission_data_KerasClassifier_2.csv', index=True)